In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re, json, itertools
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')

In [ ]:
df = pd.read_excel('posts.xlsx')

In [ ]:
df.shape

(39362, 22)

In [ ]:
df = df[['Attribute:Body', 'Attribute:Tags']]
df.columns = ['Body', 'Tags']
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(inplace=True)
df = df.iloc[:10000, :]

In [ ]:
df.head()

,Body,Tags
0,<p>I often hear about subatomic particles havi...,<quantum-mechanics><particle-physics><angular-...
1,<p>How would you explain string theory to non-...,<string-theory><education>
2,<p>This is a question that has been posted at ...,<particle-physics><group-theory><representatio...
3,<p>What are the main problems that we need to ...,<quantum-mechanics><quantum-interpretations><h...
4,<p>Hamilton's principle states that a dynamic ...,<lagrangian-formalism><variational-principle><...


In [ ]:
#preprocessing the Body
def clean_text(sentence):
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, '', sentence)
    pattern = re.compile(r'[^a-z]+')
    sentence = sentence.lower()
    sentence = pattern.sub(' ', sentence).strip()
    word_list = word_tokenize(sentence)
#     stopwords_list = set(stopwords.words('english'))
#     word_list = [word for word in word_list if word not in stopwords_list]
#     word_list = [word for word in word_list if len(word)>1]
#     ps = PorterStemmer()
#     word_list = [ps.stem(word) for word in word_list]
    sentence = ' '.join(word_list)
    return sentence
df['Body'] = df['Body'].apply(lambda x: clean_text(x))

In [ ]:
df.head()

,Body,Tags
0,i often hear about subatomic particles having ...,<quantum-mechanics><particle-physics><angular-...
1,how would you explain string theory to non phy...,<string-theory><education>
2,this is a question that has been posted at man...,<particle-physics><group-theory><representatio...
3,what are the main problems that we need to sol...,<quantum-mechanics><quantum-interpretations><h...
4,hamilton s principle states that a dynamic sys...,<lagrangian-formalism><variational-principle><...


In [ ]:
#creating one-hot representation for all the tags
df_tags = pd.DataFrame(np.zeros((df.shape[0], len_tags), dtype=int), columns=tags)
for i in range(len_rows):
    for x in df.Tags[i].split():
        df_tags[x][i] = 1
df = pd.concat([df, df_tags], axis=1)
df.head()

,Body,Tags,1pi-effective-action,absolute-units,absorption,acceleration,accelerator-physics,accretion-disk,acoustics,action,adiabatic,adm-formalism,ads-cft,aerodynamics,aether,affine-lie-algebra,air,aircraft,algebraic-geometry,algebraic-topology,algorithms,amorphous-solids,analyticity,angular-momentum,angular-velocity,antennas,anthropic-principle,anti-de-sitter-spacetime,anticommutator,antimatter,anyons,applied-physics,approximations,arrow-of-time,asteroids,astrometrics,astronomy,astrophotography,astrophysics,asymptotics,...,variational-calculus,variational-principle,vector-fields,vectors,velocity,vibrations,virial-theorem,virtual-particles,viscosity,visible-light,vision,visualization,voltage,volume,vortex,warp-drives,water,wave-particle-duality,wavefunction,wavefunction-collapse,waveguide,wavelength,waves,weak-interaction,weather,weight,white-dwarfs,white-holes,wick-rotation,wick-theorem,wightman-fields,wigner-eckart,wigner-transform,wilson-loop,wimps,work,wormholes,x-ray-crystallography,x-rays,yang-mills
0,i often hear about subatomic particles having ...,quantum-mechanics particle-physics angular-...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,how would you explain string theory to non phy...,string-theory education,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,this is a question that has been posted at man...,particle-physics group-theory representatio...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,what are the main problems that we need to sol...,quantum-mechanics quantum-interpretations h...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,hamilton s principle states that a dynamic sys...,lagrangian-formalism variational-principle ...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#list of all tags
def clean_tag(line):
    line = re.sub('[<>]', ' ', line)
    return line
df.Tags = df.Tags.apply(lambda x: clean_tag(x))
vectorizer = CountVectorizer(tokenizer= lambda text : text.split(" "))
tag_dtm = vectorizer.fit_transform(df["Tags"])
tags = vectorizer.get_feature_names()
tags = [x for x in tags if x]
len_rows = df.shape[0]
len_tags = len(tags)
print('Total no. of unique tags: ',len_tags)
print('dataframe size: ', len_rows)
tags_count = {}
temp_series = df_tags.sum().sort_values(ascending=False)
for i in range(len_tags):
    tags_count[temp_series.index[i]] = temp_series.values[i]

Total no. of unique tags:  723
dataframe size:  10000


In [ ]:
!wget 'https://nlp.stanford.edu/data/glove.6B.zip'
!unzip 'glove.6B.zip'

--2021-07-17 15:28:08--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-07-17 15:28:08--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1      100%[===================>] 822.24M  5.03MB/s    in 2m 40s  

2021-07-17 15:30:48 (5.14 MB/s) - ‘glove.6B.zip.1’ saved [862182613/862182613]

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: glove.

In [ ]:
embeddings_index = {}
f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
tags[:10]

['1pi-effective-action',
 'absolute-units',
 'absorption',
 'acceleration',
 'accelerator-physics',
 'accretion-disk',
 'acoustics',
 'action',
 'adiabatic',
 'adm-formalism']

In [ ]:
#getting tag embeddings
tag_embeddings = []
count = 0
for tag in tags:
    sum = np.zeros(50)
    try:
        sub_tags = tag.split('-')
        for x in sub_tags:
            try:
                sum += embeddings_index[x]
            except:
                continue
        tag_embeddings.append(sum/len(sub_tags))
    except:
        count+=1
        print(tag)
tag_embeddings = np.array(tag_embeddings)

In [ ]:
df.head()

,Body,Tags,1pi-effective-action,absolute-units,absorption,acceleration,accelerator-physics,accretion-disk,acoustics,action,adiabatic,adm-formalism,ads-cft,aerodynamics,aether,affine-lie-algebra,air,aircraft,algebraic-geometry,algebraic-topology,algorithms,amorphous-solids,analyticity,angular-momentum,angular-velocity,antennas,anthropic-principle,anti-de-sitter-spacetime,anticommutator,antimatter,anyons,applied-physics,approximations,arrow-of-time,asteroids,astrometrics,astronomy,astrophotography,astrophysics,asymptotics,...,variational-calculus,variational-principle,vector-fields,vectors,velocity,vibrations,virial-theorem,virtual-particles,viscosity,visible-light,vision,visualization,voltage,volume,vortex,warp-drives,water,wave-particle-duality,wavefunction,wavefunction-collapse,waveguide,wavelength,waves,weak-interaction,weather,weight,white-dwarfs,white-holes,wick-rotation,wick-theorem,wightman-fields,wigner-eckart,wigner-transform,wilson-loop,wimps,work,wormholes,x-ray-crystallography,x-rays,yang-mills
0,i often hear about subatomic particles having ...,quantum-mechanics particle-physics angular-...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,how would you explain string theory to non phy...,string-theory education,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,this is a question that has been posted at man...,particle-physics group-theory representatio...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,what are the main problems that we need to sol...,quantum-mechanics quantum-interpretations h...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,hamilton s principle states that a dynamic sys...,lagrangian-formalism variational-principle ...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# !pip install stellargraph

In [ ]:
#importing libraries
import tensorflow as tf
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.activations import sigmoid
from keras.layers import Input, Dense, Flatten, Dropout, Embedding, GlobalMaxPooling1D, dot, Reshape, Layer
from tensorflow.keras.backend import variable, dot as k_dot, sigmoid, relu
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow import Variable

from sklearn.model_selection import train_test_split

### Embedding of questions

In [ ]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def max_length(lines):
    return max([len(s.split()) for s in lines])

def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [ ]:
X = df['Body']
y = df.drop(['Body', 'Tags'], axis=1)

In [ ]:
tokenizer = create_tokenizer(X)
length = max_length(X)
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

Max document length: 3325
Vocabulary size: 24280


In [ ]:
X = encode_text(tokenizer, X, length)
print(X.shape)

(10000, 3325)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Adjacency Matrix

In [ ]:
adj_matrix_path = 'adjacency_matrix.json'
# Count all labels.
nums = np.sum(np.array(df_tags), axis=0)
adj = np.zeros((len_tags, len_tags), dtype=int)
# Now iterate over the whole training set and consider all pairs of labels in sample annotation.
for sample in np.array(df_tags):
    sample_idx = np.argwhere(sample > 0)[:, 0]
    # We count all possible pairs that can be created from each sample's set of labels.
    for i, j in itertools.combinations(sample_idx, 2):
        adj[i, j] += 1
        adj[j, i] += 1
# Save it for further use.        
with open(adj_matrix_path, 'w') as fp:
    json.dump({
        'nums': nums.tolist(),
        'adj': adj.tolist()
    }, fp, indent=3)

In [ ]:
# GCNConv.preprocess(adj).astype('f4')
tag_embeddings.shape

(723, 50)

### Defining Model

In [ ]:
def custom_model(length, vocab_size, tag_embeddings, adj):
    inputs1 = Input(shape=(length, ))
    embedding = Embedding(vocab_size, 50)(inputs1)
    conv = Conv1D(filters=50, kernel_size=4, activation='relu')(embedding)
    pool = GlobalMaxPooling1D()(conv)
    model = Model(inputs1, pool)
    print(model.summary())
    
#     X_in = Input(shape=(len(tag_embeddings[0]), ))
# #     fltr_in = Input((len(tag_embeddings), ))
      
#     graph_conv_1 = GCNConv(50, activation='relu')([X_in, adj])
#     graph_conv_2 = GCNConv(50, activation='relu')([graph_conv_1, adj])
#     print(pool.shape)
#     print(graph_conv_2.shape)
#     output = tf.matmul(pool, tf.transpose(graph_conv_2))
#     output = sigmoid(output)
#     print(output.shape)
    
#     gcn = Model(inputs = [inputs1, X_in], outputs = output)
#     print(gcn.summary())
#     gcn.compile(optimizer='adam', loss='binary_crossentropy', weighted_metrics=['acc'])
#     gcn.fit([X, tag_embeddings] , y.values, epochs=10)
custom_model(length, vocab_size, tag_embeddings, adj)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3325)]            0         
_________________________________________________________________
embedding (Embedding)        (None, 3325, 50)          1214000   
_________________________________________________________________
conv1d (Conv1D)              (None, 3322, 50)          10050     
_________________________________________________________________
global_max_pooling1d (Global (None, 50)                0         
Total params: 1,224,050
Trainable params: 1,224,050
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
print(length)
print(vocab_size)

3325
24280


### Pytorch Implementation

In [ ]:
import torch
import torch.nn as nn
import math

In [ ]:
device = torch.device('cuda')
device
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(np.array(y_train))
X_val = torch.from_numpy(X_val)
y_val = torch.from_numpy(np.array(y_val))
tag_embeddings = torch.from_numpy(tag_embeddings)

In [ ]:
# function for creating adjacency matrix (over-smoothing problem is also taken into consideration)
'''
num_classes ---> no. of unique tags
t ---> threshold for binarizing the correlation matrix.
p ---> determines the weights assigned to a node itself and other correlated nodes
adj_data ---> contains co-occurrence matrix and corresponding tag counts
    
*Note -> for info. regarding all these parameters go to https://arxiv.org/pdf/1904.03582.pdf
'''

t = 0.4
p = 0.2
def gen_A(len_tags, t, p, adj_data):
    adj = np.array(adj_data['adj']).astype(np.float32)
    nums = np.array(adj_data['nums']).astype(np.float32)
    nums = nums[:, np.newaxis]
    adj = adj / nums
    adj[adj < t] = 0
    adj[adj >= t] = 1
    adj = adj * p / (adj.sum(0, keepdims=True) + 1e-6)  
    adj = adj + np.identity(len_tags, np.int)
    return adj

#normalization of Adjacency matrix
def gen_adj(A):
    D = torch.pow(A.sum(1).float(), -0.5)
    D = torch.diag(D).type_as(A)
    adj = torch.matmul(torch.matmul(A, D).t(), D)
    return adj

In [ ]:
#defining Graph Convolution Network
"""
Simple GCN layer (pytorch), similar to https://arxiv.org/abs/1609.02907

"""
class GraphConvolution(nn.Module):
    
    def __init__(self, in_features, out_features, bias=False):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(in_features, out_features), requires_grad=True)
        if bias:
            self.bias = nn.Parameter(torch.Tensor(1, 1, out_features), requires_grad=True)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.matmul(input.float(), self.weight)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


class TagRetrieval(nn.Module):
    
    def __init__(self, len_tags, adj_matrix_path, vocab_size, embedding_dim, t=0.1, p=0.25):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.conv1d = nn.Conv1d(embedding_dim, embedding_dim, 4, padding=0)
        self.globalmaxpool1d = nn.AdaptiveMaxPool2d(output_size=(embedding_dim, 1))
        
        self.gc1 = GraphConvolution(50, 50)
        self.gc2 = GraphConvolution(50, 50)
        self.relu = nn.LeakyReLU(0.2)
        self.sigm = nn.Sigmoid()
        # Load data for adjacency matrix
        with open(adj_matrix_path) as fp:
            adj_data = json.load(fp)
        # Compute adjacency matrix
        adj = gen_A(len_tags, t, p, adj_data)
        self.A = nn.Parameter(torch.from_numpy(adj).float(), requires_grad=False)
        
    def forward(self, question, tags_embeddings):
        # Get visual features from image
        #feature = self.features(imgs)
        #feature = feature.view(feature.size(0), -1)
        embedding = self.embedding(question)
        embedding = embedding.permute([0, 2, 1])
        conv1d = self.conv1d(embedding)
        globalmaxpool1d = self.globalmaxpool1d(conv1d)
        feature = globalmaxpool1d.squeeze()
        # Get graph features from graph
        #inp = inp[0].squeeze()
        inp = tag_embeddings
        adj = gen_adj(self.A).detach()
        x = self.gc1(inp, adj)
        x = self.relu(x)
        x = self.gc2(x, adj)
        
        # We multiply the features from GСN and СNN in order to take into account 
        # the contribution to the prediction of classes from both the image and the graph.
        x = x.transpose(0, 1)
        x = torch.matmul(feature, x)
        return self.sigm(x)
    
model = TagRetrieval(len_tags, adj_matrix_path, vocab_size, 50)
# model = model.to(device)
# print(model(X_train[:1000, :], tag_embeddings).shape)
    
print(model.parameters)


<bound method Module.parameters of TagRetrieval(
  (embedding): Embedding(24280, 50, padding_idx=0)
  (conv1d): Conv1d(50, 50, kernel_size=(4,), stride=(1,))
  (globalmaxpool1d): AdaptiveMaxPool2d(output_size=(50, 1))
  (gc1): GraphConvolution (50 -> 50)
  (gc2): GraphConvolution (50 -> 50)
  (relu): LeakyReLU(negative_slope=0.2)
  (sigm): Sigmoid()
)>


In [ ]:
# X_train = X_train.to(device)
# y_train = y_train.to(device)
# X_val = X_val.to(device)
# y_val = y_val.to(device)
# tag_embeddings = tag_embeddings.to(device)

In [ ]:
X_train.device

device(type='cpu')

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
for epoch in range(5):
  optimizer.zero_grad()
  model_result = model(X_train, tag_embeddings)
  loss = criterion(model_result, y_train.float())
  print('Epoch no: {} and the loss : {}'.format(epoch+1, loss))
  loss.backward()
  optimizer.step()

In [ ]:
x = torch.tensor([[1,2, 12,34, 56,78, 90,80],
                 [12,45, 99,67, 6,23, 77,82],
                 [3,24, 6,99, 12,56, 21,22]])
embed = nn.Embedding(3325, 50, padding_idx=0)
print(x.shape)
x = embed(x)
print(x.shape)
x = x.permute([0, 2, 1])
print(x.shape)
conv1d = nn.Conv1d(50, 50, 4, padding=0)
x = conv1d(x)
print(x.shape)
globalmaxpool1d = nn.AdaptiveMaxPool2d(output_size=(50, 1))
x = globalmaxpool1d(x)
print(x.shape)
x = x.squeeze()
print(x.shape)


gc1 = GraphConvolution(50, 50)
y = gc1(torch.from_numpy(tag_embeddings), A)
print(y.shape)
gc2 = GraphConvolution(50, 50)
y = gc2(y, A)
print(y.shape)
y = y.transpose(0, 1)
print(y.shape)
y = torch.matmul(x, y)
print(y.shape)
sigm = nn.Sigmoid()
y = sigm(y)
print(y.shape)

torch.Size([3, 8])
torch.Size([3, 8, 50])
torch.Size([3, 50, 8])
torch.Size([3, 50, 5])
torch.Size([3, 50, 1])
torch.Size([3, 50])
torch.Size([723, 50])
torch.Size([723, 50])
torch.Size([50, 723])
torch.Size([3, 723])
torch.Size([3, 723])


In [ ]:
with open(adj_matrix_path) as fp:
    adj_data = json.load(fp)
# Compute adjacency matrix
adj = gen_A(len_tags, t, p, adj_data)
A = nn.Parameter(torch.from_numpy(adj).float(), requires_grad=False)
A.shape

torch.Size([723, 723])

In [ ]:
A

Parameter containing:
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [ ]:
type(torch.from_numpy(tag_embeddings))

torch.Tensor

In [ ]:
X_train[:100, :].shape

(100, 3325)

In [ ]:
X_train.shape

torch.Size([8000, 3325])

In [ ]:
X_train[:5000].shape

torch.Size([5000, 3325])